In [1]:
import grape
import pandas as pd

2024-01-22 13:31:10.404754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:pykeen.utils:Using opt_einsum


In [2]:
from grape.datasets.kgobo import NCBITAXON

In [3]:
_ = NCBITAXON()

In [4]:
ncbi = pd.read_csv("./graphs/kgobo/NCBITAXON/2023-06-16/ncbitaxon_kgx_tsv/ncbitaxon_kgx_tsv_nodes.tsv", sep="\t", low_memory=False)

In [5]:
species = pd.read_csv("./data/molecules/230106_frozen_metadata.csv.gz", low_memory=False)

In [6]:
species['wd_taxon'] = 'wd:' + species['organism_wikidata'].str.extract(r"(Q\d+)")

In [7]:
species = species.dropna(subset='organism_taxonomy_ncbiid').drop_duplicates(subset='organism_taxonomy_ncbiid')

In [8]:
wd_to_ncbi_edges=pd.DataFrame({
    'wikidata': species['wd_taxon'],
    'ncbi': [f"NCBITaxon:{i}" for i in species['organism_taxonomy_ncbiid'].astype(int).values],
    'type': 'biolink:same_as',
})

In [9]:
ncbi_node = pd.concat(
    [
        pd.DataFrame({
            'node': wd_to_ncbi_edges.wikidata,
            'type': 'biolink:OrganismTaxon',
        }),
        pd.DataFrame({
            'node': wd_to_ncbi_edges.ncbi,
            'type': 'biolink:OrganismalEntity'
        })
    ]
).drop_duplicates()

In [10]:
from grape import Graph

In [11]:
wd_to_ncbi_graph = Graph.from_pd(
    directed=True,
    edges_df=wd_to_ncbi_edges,
    edge_src_column='wikidata',
    edge_dst_column='ncbi',
    edge_type_column='type',
    nodes_df=ncbi_node,
    node_name_column='node',
    node_type_column='type'
)

In [12]:
lotus = Graph.from_csv(
    name="LOTUS",
    node_path="./data/full_lotus_nodes.csv",
    node_list_separator="\t",
    node_list_header=True,
    nodes_column_number=0,
    node_list_node_types_column_number=1,
    edge_path="./data/full_lotus_edges.csv",
    edge_list_separator="\t",
    edge_list_header=True,
    sources_column_number=0,
    destinations_column_number=1,
    edge_list_edge_types_column_number=2,
    #weights_column_number=3,
    directed=True,
)

In [13]:
ncbi_graph = NCBITAXON()

In [14]:
lotus_with_ncbi = lotus|wd_to_ncbi_graph|ncbi_graph.to_directed()

In [15]:
lotus_with_ncbi.dump_nodes(
    path="./data/lotus_with_ncbi_nodes.csv",
    header=True,
    nodes_column_number=0,
    nodes_column="nodes",
    node_types_column_number=1,
    node_type_column="type"
)

In [16]:
lotus_with_ncbi.dump_edges(
    path="./data/lotus_with_ncbi_edges.csv",
    header=True,
    directed=True,
    edge_types_column_number=2,
    edge_type_column='edge_type',
)

In [17]:
Graph.from_csv(
    name="LOTUS_with_NCBITaxon",
    node_path="./data/lotus_with_ncbi_nodes.csv",
    node_list_separator="\t",
    node_list_header=True,
    nodes_column_number=0,
    node_list_node_types_column_number=1,
    edge_path="./data/lotus_with_ncbi_edges.csv",
    edge_list_separator="\t",
    edge_list_header=True,
    sources_column_number=0,
    destinations_column_number=1,
    edge_list_edge_types_column_number=2,
    directed=True,
    #directed=False,
)

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>LOTUS_with_NCBITaxon</h2><p class="paragraph">The directed graph LOTUS_with_NCBITaxon has 2.79M heterogeneous nodes and 6.07M heterogeneous edges. The RAM requirements for the nodes and edges data structures are 328.83MB and 46.56MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 0, the maximum node degree is 43.71K, the mode degree is 1, the mean degree is 2.18 and the node degree median is 1.</p><p class="paragraph">The nodes with the highest degree centrality are <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=500585' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:500585'>NCBITaxon:500585</a> (degree 43.71K and node type <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a>), <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=329961' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:329961'>NCBITaxon:329961</a> (degree 43.41K and node type <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a>), <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=114727' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:114727'>NCBITaxon:114727</a> (degree 36.13K and node type <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a>), <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=119210' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:119210'>NCBITaxon:119210</a> (degree 35.31K and node type <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a>) and <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=49928' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:49928'>NCBITaxon:49928</a> (degree 33.68K and node type <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a>).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 4 node types, which are <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a> (2.51M nodes, 90.09%), <a href='https://biolink.github.io/biolink-model/docs/ChemicalEntity.html' target='_blank' title='Go to BioLink to get m